## Extraction: Web Crawler

Extract coronavirus data from the primary site, using it as one of the disperate sources to initiate the Extraction, Transformation and Loading (ETL) process. This is the initial extract.

In [133]:
#import the requests library and download the webpage of interest

import requests
page=requests.get("https://www.worldometers.info/coronavirus/#countries")
page

<Response [200]>

In [135]:
#import the BeautifulSoup library to parse the conetnts of downloaded page

from bs4 import BeautifulSoup
soup=BeautifulSoup(page.content, 'html.parser')

#find tag of interest using id/css selectors 
corvid_details= soup.find(id='main_table_countries_today')
actuals=corvid_details.find('tbody')

In [136]:
#create empty lists which will be used to append the row(tr) and cell details(td) from the downloaded page
#the website's owners keep updating it, kindly inspect the elements referenced if you encounter any error

A=[]
B=[]
C=[]
D=[]
E=[]
F=[]
G=[]
H=[]
I=[]
J=[]

for row in actuals.find_all("tr"):
    cells=row.find_all('td')
    if len(cells)==10:
        A.append(cells[0].find(text=True))  
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        D.append(cells[3].find(text=True))
        E.append(cells[4].find(text=True))
        F.append(cells[5].find(text=True))
        G.append(cells[6].find(text=True)) 
        H.append(cells[7].find(text=True))
        I.append(cells[8].find(text=True))
        J.append(cells[9].find(text=True))

In [137]:
#import the pandas library

import pandas as pd

#combine the data into a pandas dataframe

cases=pd.DataFrame(A, columns=['Country, Other'])
cases['Total Cases']=B
cases['New Cases']=C
cases['Total Deaths']=D
cases['New Deaths']=E
cases['Total Recovered']=F
cases['Active Cases']=G
cases['Serious/Critical']=H
cases['Tot Cases/1M pop']=I
cases['Tot Deaths/1M pop']=J

## Transformation: Derived Column Transformation

This stage marks our initial transformation where we will derive a new date column and add it to to our dataframe

In [138]:
#import the datetime library to derive date details as a string in your local time

import datetime as dt

In [139]:
#add the Date column to the data in our cases dataframe

cases['Date']= dt.datetime.today().strftime("%m/%d/%Y")

In [140]:
#verify whether the correct date has been captured

cases.head()

,"Country, Other",Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,Serious/Critical,Tot Cases/1M pop,Tot Deaths/1M pop,Date
0,USA,"245,373",+496,"6,095",+25,"10,403","228,875","5,421",741,18,04/03/2020
1,Italy,"115,242",None,"13,915",None,"18,278","83,049","4,053","1,906",230,04/03/2020
2,Spain,"112,065",None,"10,348",None,"26,743","74,974","6,092","2,397",221,04/03/2020
3,Germany,"84,794",None,"1,107",None,"22,440","61,247","3,936","1,012",13,04/03/2020
4,France,"59,105",None,"5,387",None,"12,428","41,290","6,399",905,83,04/03/2020


In [141]:
#write to excel
#take note of the naming convention

cases.to_excel('cases_04_03_eod.xlsx', index=False)

## Transformation: Union Transformation

This stage marks our final transformation where will union all the excel files we have generated over time to track the coronavirus. In our case we have data we collected on the 2nd of April and the 3rd of April and which follow a common naming convention

In [142]:
#import the os and re libraries/modules

import os, re

In [143]:
#create an empty dataframe

historical_cases_data= pd.DataFrame()

In [144]:
#create a list of the files in the directory

case_files=os.listdir('.')

#### Union Transformation Steps: 

1. Loop through the files to read them into the staging_data dataframe
2. Leverage the re library to specify files that start with/match the naming convention used ('cases_')
3. Append the data in the staging_data dataframe to the empty dataframe created initially (Union Transform)
4. Validate the final dataset using the head and tail functions

In [145]:
for file in case_files:
    if re.match('cases_', file):
        staging_data=pd.read_excel(file, index_col=[0])
        historical_cases_data=historical_cases_data.append(staging_data)

In [146]:
#use the head function to return the first 5 records

historical_cases_data.head()

,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,Serious/Critical,Tot Cases/1M pop,Tot Deaths/1M pop,Date
"Country, Other",,,,,,,,,,
USA,"215,357",+354,"5,113",11.0,"8,878","201,366","5,005",651,15.0,04/02/2020
Italy,"110,574",NaN,"13,155",NaN,"16,847","80,572","4,035","1,829",218.0,04/02/2020
Spain,"110,238","+6,120","10,003",616.0,"26,743","73,492","6,092","2,358",214.0,04/02/2020
Germany,"79,465","+1,484",959,28.0,"19,175","59,331","3,408",948,11.0,04/02/2020
France,"56,989",NaN,"4,032",NaN,"10,935","42,022","6,017",873,62.0,04/02/2020


In [147]:
#use the tail function to return the last 5 records

historical_cases_data.tail()

,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,Serious/Critical,Tot Cases/1M pop,Tot Deaths/1M pop,Date
"Country, Other",,,,,,,,,,
St. Vincent Grenadines,2,NaN,,NaN,1,1,NaN,18,NaN,04/03/2020
Sierra Leone,2,NaN,,NaN,NaN,2,NaN,0.3,NaN,04/03/2020
Papua New Guinea,1,NaN,,NaN,NaN,1,NaN,0.1,NaN,04/03/2020
Timor-Leste,1,NaN,,NaN,NaN,1,NaN,0.8,NaN,04/03/2020
China,"81,620",31,"3,322",4.0,"76,571","1,727",379,57,2.0,04/03/2020
